In [5]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap

c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
ts= pd.read_csv('../../data/ts_top100_prediction.csv')

(1739323, 5)


,date,item,sales,income,event
1468162,2015-07-28,SUPERMARKET_3_090,38,62.928,0


In [11]:
print(ts.shape)
ts.sample(1)

(1739323, 5)


,date,item,sales,income,event
596905,2013-02-15,SUPERMARKET_1_170,3,21.096,0


In [13]:
ts['date']= pd.to_datetime(ts['date'], format='%Y-%m-%d')

In [14]:
ts['weekday'] = ts['date'].dt.day_name()

In [15]:
ts.sample(5)

,date,item,sales,income,event,weekday
206240,2011-11-17,SUPERMARKET_3_516,0,0.000,0,Thursday
1413006,2015-06-03,SUPERMARKET_2_056,5,41.880,0,Wednesday
316152,2012-04-02,SUPERMARKET_3_607,16,43.776,0,Monday
1389332,2015-05-10,SUPERMARKET_2_183,0,0.000,0,Sunday
1074234,2014-06-27,SUPERMARKET_3_753,0,0.000,0,Friday


In [16]:
lista_productos100 = list(ts['item'].unique())

In [17]:
resultados = []

# Copia del DataFrame original
df_original = ts.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for i in lista_productos100:
        dataset = df_original[['item','date','sales','weekday','event']]
        df_product = dataset[dataset['item']==i]
        df_product_sin_duplicates = df_product[['date','weekday', 'event']].drop_duplicates()
        ts_predict = df_product.groupby(['date'])['sales'].sum().reset_index()
        ts_predict = ts_predict.merge(df_product_sin_duplicates, on=['date'], how='left')
        ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
        ts_predict['date'] = pd.to_datetime(ts_predict['date'])
        ts_predict.sort_values('date', ascending=True, inplace=True)
        ts_predict.set_index('date', inplace=True)
        steps = 30
        ts_predict = ts_predict.asfreq('D')
        y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
        y_train, y_test = y[:-30], y[-30:]
        exog_train, exog_test = exog[:-30], exog[-30:]        
        forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123,
                                                        max_depth=None,
                                                        min_samples_leaf=2, 
                                                        min_samples_split=2,
                                                        n_estimators=50),
                    lags      =  14
                )
        
        start_date = '2016-04-25' 
        end_date ='2016-05-30'
        # Crear un rango de fechas
        date_range = pd.date_range(start=start_date, end=end_date)
        # Crear el dataframe
        df1 = pd.DataFrame(date_range, columns=['date'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1['weekday'] = df1['date'].dt.day_name()
        df1['event'] = 0
        df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
        df1.sort_values('date', inplace=True, ascending=True)
        df1.set_index('date', inplace=True)
        df1 = df1.asfreq('D')
        exog_test = pd.concat([exog_test, df1], ignore_index=False)
        forecaster.fit(y=y_train, exog=exog_train)
        predictions = forecaster.predict(steps=60, exog=exog_test).to_list()
        resultados.append({'i':i, 'prediction':predictions,'test':y_test.values})

c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\harol\Nuclio\DS-MARKET-TFM\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
c:\Users\har

In [ ]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediction'] = tiendas_inventario['prediction'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['i','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

In [26]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

240920
239651.0
1269.0


In [ ]:
units_globales = pd.DataFrame(resultados) 

In [17]:
file_name = "productos_global_unidades_vendidas.xlsx"
units_globales.to_excel(file_name)